In [15]:
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import warnings
import math
from scipy.optimize import curve_fit
from collections import deque, defaultdict
from itertools import combinations
import seaborn as sns
from scipy import stats
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [16]:
# Data extractor functions
def connectivityExtractor(name, pruned):
    if pruned == 0:
        file_path = 'Networks/Network_Vessels_' + name +'.mat'
    elif pruned == 1: 
        file_path = 'Pruned/Pruned_Network_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    connectivity_raw = data_structure['connectivity'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    connectivity_data = connectivity_raw.squeeze()
    # Create a DataFrame from the connectivity data
    connectivity_df = pd.DataFrame(connectivity_data, columns=['Parent', 'Daughter1', 'Daughter2', 'Daughter3'])
    connectivity_df.replace(0, np.nan, inplace=True) #ensure all nonexistent vessels have NaN
    connectivity_df.at[0,'Parent']=0 #make sure first vessel is 0 (purposefully removed in last step for ease)
    # Save the DataFrame to inspect it
    return connectivity_df

def nodesExtractor(name, pruned): #extracts nodes and their corresponding information
    if pruned == 0:
        file_path = 'Networks/Network_Vessels_' + name +'.mat'
    elif pruned == 1: 
        file_path = 'Pruned/Pruned_Network_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    if pruned == 0:
        data_structure = matlab_data['nodesC2']
    elif pruned == 1:
        data_structure = matlab_data['nodesC3']
    # Reshape or ensure it's a proper 2D array (if required)
    nodes_data = data_structure.squeeze()
    # Create a DataFrame from the connectivity data
    nodes_df = pd.DataFrame(nodes_data, columns=['NodeID', 'X', 'Y', 'Z', 'Degree'])
    # Save the DataFrame to inspect it
    return nodes_df

def edgesExtractor(name, pruned): #extracts segments to create a dataframe of from and to nodes
    if pruned == 0:
        file_path = 'Networks/Network_Vessels_' + name +'.mat'
    elif pruned == 1: 
        file_path = 'Pruned/Pruned_Network_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'segments' field
    data_structure = matlab_data['segments']
    # Reshape or ensure it's a proper 2D array (if required)
    edges_data = data_structure.squeeze()
    # Create a DataFrame from the connectivity data
    edge_df = pd.DataFrame(edges_data, columns=['ID', 'From', 'To'])
    # Save the DataFrame to inspect it
    return edge_df
    
def findInputVessel(segments,fromnode,to):
    vessel = segments[((segments['From'] == fromnode)&(segments['To']==to))|((segments['From'] == to)&(segments['To']==fromnode))]
    return int(vessel['ID'].iloc[0])

def mapIDExtractor(name, pruned):
    if pruned == 0:
        file_path = 'Networks/Network_Vessels_' + name +'.mat'
    elif pruned == 1: 
        file_path = 'Pruned/Pruned_Network_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'mapID' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    map_raw = data_structure['mapIDs'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    map_data = map_raw.squeeze()
    # Create a DataFrame from the connectivity data
    map_df = pd.DataFrame(map_data, columns=['New', 'ID'])
    # Save the DataFrame to inspect it
    return map_df

def lobeExtractor(name, vesID,pruned):
    data = connectivityExtractor(name,pruned)
    
    tree = defaultdict(list)
    for _,row in data.iterrows():
        parent = row['Parent']
        for daughter_col in ['Daughter1','Daughter2','Daughter3']:
            daughter = row[daughter_col]
            if pd.notna(daughter):
                tree[parent].append(daughter)

    visited = set()
    queue = deque([vesID])

    while queue:
        current = queue.popleft()
        if current not in visited:
            visited.add(current)
            queue.extend(tree.get(current,[]))
    
    visited.discard(vesID)  # Remove vesID from visited
    downstream_df = data[data['Parent'].isin(visited)]
    return downstream_df

def node_loc(name,lobe_nodes,pruned):
    nodes = nodesExtractor(name,pruned)
    lobe = nodes[nodes['NodeID'].isin(lobe_nodes)]
    return lobe[['X','Y','Z']]

def saveLobeStatistics(name,fromnode,tonode,pruned):
    segments = edgesExtractor(name, pruned)
    maps = mapIDExtractor(name, pruned)
    vesID = findInputVessel(segments,fromnode,tonode)
    newID = int(maps[maps['Old']==vesID]['New'].iloc[0])
    lobe_ves = lobeExtractor(name,newID, pruned)
    new_lobe_ves_ID = lobe_ves['Parent'].to_numpy()
    oldID = maps[maps['New'].isin(new_lobe_ves_ID)]['Old'].to_numpy()
    fromnodes = segments[segments['ID'].isin(oldID)]['From'].to_numpy()
    tonodes = segments[segments['ID'].isin(oldID)]['To'].to_numpy()
    lobe_nodes = np.unique(np.concatenate((fromnodes,tonodes))).astype(int)
    lobe_node_loc = node_loc(name,lobe_nodes,pruned)/1000
    return lobe_node_loc.to_numpy()

def twoInputVessels(name,from1,to1,from2,to2,pruned):
    points1 = saveLobeStatistics(name,from1,to1,pruned)
    points2 = saveLobeStatistics(name,from2,to2,pruned)
    return np.unique(np.concatenate((points1,points2),axis=0),axis=0)

In [17]:
def lengthExtractor(name, pruned):
    if pruned == 0:
        file_path = 'Networks/Network_Vessels_' + name +'.mat'
    elif pruned == 1: 
        file_path = 'Pruned/Pruned_Network_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'lengths' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    length_raw = data_structure['lengths'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    length_data = length_raw.squeeze()
    # Create a DataFrame from the data
    length_df = pd.DataFrame(length_data, columns=['Length'])
    # Save the DataFrame to inspect it
    return length_df

def radiusExtractor(name, pruned):
    if pruned == 0:
        file_path = 'Networks/Network_Vessels_' + name +'.mat'
    elif pruned == 1: 
        file_path = 'Pruned/Pruned_Network_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'radius' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    radius_raw = data_structure['rin'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    radius_data = radius_raw.squeeze()
    # Create a DataFrame from the data
    radius_df = pd.DataFrame(radius_data, columns=['Radius'])
    # Save the DataFrame to inspect it
    return radius_df

def volumeExtractor(name,pruned):
    if pruned == 0:
        file_path = 'Networks/Network_Vessels_' + name +'.mat'
    elif pruned == 1: 
        file_path = 'Pruned/Pruned_Network_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    volume_raw = matlab_data['volumes']  # This is an x by 1 or 1D array
    volume_data = volume_raw.squeeze()  # Converts to 1D array if it's still 2D with 1 column
    volume_df = pd.DataFrame(volume_data, columns=['Volume'])
    return volume_df

def lRR(data):
    lrr = np.empty((0,0))
    for i in np.arange(data.shape[0]): #iterate over every vessel
        length = data.iloc[i]['Length'] #pull the length value
        radius = data.iloc[i]['Radius'] #retreive radius value
        lrr_value = length/radius #calculate lrr
        lrr = np.append(lrr,lrr_value) #append lrr value to total list
    data['Length/Radius'] = lrr.tolist() #add list to table
    return data

def assignDepth(data):
    tree = {} 
    for _, row in data.iterrows(): #iterate over every row
        parent = row['Parent']
        daughters = row[['Daughter1', 'Daughter2', 'Daughter3']].dropna().astype(int).tolist() #only select daughter values that exist
        tree[parent] = daughters
    # Step 2: BFS traversal to assign depths
    depths = {}
    queue = deque([(data['Parent'].iloc[0], 0)])  # Start from root (first parent)

    while queue:
        node, depth = queue.popleft()
        depths[node] = depth
        for child in tree.get(node, []):
            if child not in depths:  # Avoid revisiting
                queue.append((child, depth + 1))
    data['Depth'] = data['Parent'].map(depths)
    return data

def lengthFromRoot(data):
    tree = {}
    length_dict = dict(zip(data['Parent'], data['Length']))  # Map each vessel (parent) to its length

    for _, row in data.iterrows():
        parent = row['Parent']
        daughters = row[['Daughter1', 'Daughter2', 'Daughter3']].dropna().astype(int).tolist()
        tree[parent] = daughters

    # BFS traversal to calculate cumulative lengths
    lengths = {}
    root = data['Parent'].iloc[0]
    queue = deque([(root, 0)])  # Start with cumulative length 0

    while queue:
        node, cum_length = queue.popleft()
        lengths[node] = cum_length
        for child in tree.get(node, []):
            if child not in lengths:  # Avoid revisiting
                additional_length = length_dict.get(child, 0)
                queue.append((child, cum_length + additional_length))

    data['LengthFromRoot'] = data['Parent'].map(lengths)
    return data

def tortuosityCalculator(data,name,pruned):
    nodes = nodesExtractor(name,pruned)
    edges = edgesExtractor(name,pruned)
    maps = mapIDExtractor(name,pruned)
    tortuosity = np.empty((0,0))
    for i in np.arange(data.shape[0]):
        old_vesID = maps.iloc[i]['ID'] #find the old vessel ID
        edge_row = edges.index.get_loc(edges[edges['ID'] == old_vesID].index[0])
        node1 = edges.iloc[edge_row]['From'] #identify from node
        node2 = edges.iloc[edge_row]['To'] #identify to node
        node1_row = nodes.index.get_loc(nodes[nodes['NodeID'] == node1].index[0])
        node2_row = nodes.index.get_loc(nodes[nodes['NodeID'] == node2].index[0])
        node1_x = nodes.iloc[node1_row]['X'] #pull node 3d coordinate
        node1_y = nodes.iloc[node1_row]['Y']
        node1_z = nodes.iloc[node1_row]['Z']
        node2_x = nodes.iloc[node2_row]['X']
        node2_y = nodes.iloc[node2_row]['Y']
        node2_z = nodes.iloc[node2_row]['Z']
        euclid_dist = math.sqrt((node1_x - node2_x)**2 + (node1_y - node2_y)**2 + (node1_z - node2_z)**2) #calculate euclidean distance between from and to
        length = data.iloc[i]['Length'] #pull length and convert to mm
        ratio = length/euclid_dist #find tortuosity ratio
        tortuosity = np.append(tortuosity,ratio)
    data['Tortuosity'] = tortuosity.tolist() #add to table
    return data

def radiusFinder(data):
    dv1r = np.empty((0,0))
    dv2r = np.empty((0,0))
    dv3r = np.empty((0,0))
    for i in np.arange(data.shape[0]):# iterate over every row
        dv1 = data.iloc[i]['Daughter1'] #pull radii values
        dv2 = data.iloc[i]['Daughter2']
        dv3 = data.iloc[i]['Daughter3']
        if math.isnan(dv1): #if daughter vessel 1 doesn't exist, none of them do
            dv1r = np.append(dv1r,np.nan)
            dv2r = np.append(dv2r,np.nan)
            dv3r = np.append(dv3r,np.nan)
            continue
        dv1r = np.append(dv1r,data.iloc[int(dv1)][2])
        dv2r = np.append(dv2r,data.iloc[int(dv2)][2])
        if math.isnan(dv3): #explicitly check whetehr dv3 exist
            dv3r = np.append(dv3r,np.nan)
            continue
        dv3r = np.append(dv3r,data.iloc[int(dv3)][2])
    data['DV1R'] = dv1r.tolist()
    data['DV2R'] = dv2r.tolist()
    data['DV3R'] = dv3r.tolist()
    #select and reorder columns
    data = data.loc[:,['Parent','Depth','Length','Radius','Volume', 'Length/Radius','LengthFromRoot','Tortuosity','Daughter1','DV1R','Daughter2','DV2R','Daughter3','DV3R']]
    return data

def alphaBeta(data):
    alpha = np.empty((0,0))
    beta = np.empty((0,0))
    for i in np.arange(data.shape[0]):
        if not math.isnan(data.iloc[i]['DV3R']): #check if trifurcation, if it is then don't calculate
            alpha = np.append(alpha,np.nan)
            beta = np.append(beta,np.nan)
            continue
        dv1r = data.iloc[i]['DV1R']
        dv2r = data.iloc[i]['DV2R']
        pv = data.iloc[i]['Radius']
        if math.isnan(dv1r): #check if daughter vessels exist
            alpha = np.append(alpha,np.nan)
            beta = np.append(beta,np.nan)
            continue
        if dv1r>=dv2r: #case 1 of daughter vessels
            alpha_value = dv1r/pv
            beta_value = dv2r/pv
        else: #case 2 of daughter vessels
            alpha_value = dv2r/pv
            beta_value = dv1r/pv
        alpha = np.append(alpha,alpha_value)
        beta = np.append(beta,beta_value)
    data['Alpha'] = alpha.tolist() #add alpha and beta to total table
    data['Beta'] = beta.tolist()
    return data

def xiFinder(data):
    xi_values = np.empty((0,0))
    error_values = np.empty((0,0))
    for i in np.arange(data.shape[0]):
        rp = data.iloc[i]['Radius']/1000 #pull all radii and convert to mm
        rd1 = data.iloc[i]['DV1R']/1000
        rd2 = data.iloc[i]['DV2R']/1000
        rd3 = data.iloc[i]['DV3R']/1000
        if not math.isnan(rd3): #don't calculate if trifucation
            xi_values = np.append(xi_values,np.nan)
            error_values = np.append(error_values,np.nan)
            continue
        if math.isnan(rd1): #don't calculate if no daughter vessels
            xi_values = np.append(xi_values,np.nan)
            error_values = np.append(error_values,np.nan)
            continue
        xi = 2.5
        error = rp**xi - rd1**xi - rd2**xi
        while abs(error) > 1e-3: #allow for error up to 10^-4
            fun = rp**xi - rd1**xi - rd2**xi # solve at f(xi)
            fun2 = (rp**xi)*math.log(rp) - (rd1**xi)*math.log(rd1) - (rd2**xi)*math.log(rd2) #solve at f'(xi)
            xi = xi - fun/fun2 # new xi value
            error = rp**xi - rd1**xi - rd2**xi #calculate new error
        if xi > 4: #bound xi
            xi = 4
        if xi < 2:
            xi = 2
        error = rp**xi - rd1**xi - rd2**xi
        xi_values = np.append(xi_values,xi)
        error_values = np.append(error_values,error)
    data['Xi'] = xi_values.tolist()#add xi to the table
    #data['Error Newton'] = error_values.tolist()
    return data

def asymmetry(data):
    asym = np.empty((0,0))
    for i in np.arange(data.shape[0]):
        if not math.isnan(data.iloc[i]['DV3R']): #don't calculate if trifucation
            asym = np.append(asym,np.nan)
            continue
        dv1r = data.iloc[i]['DV1R'] #pull radii
        dv2r = data.iloc[i]['DV2R']
        if math.isnan(dv1r): #if no daughters, skip
            asym = np.append(asym,np.nan)
            continue
        if dv1r>=dv2r: #case 1
            asym_value = (dv2r/dv1r)**2
        else: #case 2
            asym_value = (dv1r/dv2r)**2
        asym = np.append(asym,asym_value)
    data['Gamma'] = asym.tolist() #add to table
    return data

def area(data):
    area = np.empty((0,0))
    for i in np.arange(data.shape[0]):
        if not math.isnan(data.iloc[i]['DV3R']): #if trifucation, skip
            area = np.append(area,np.nan)
            continue
        dv1r = data.iloc[i]['DV1R'] #pull necessary radii
        dv2r = data.iloc[i]['DV2R']
        pv = data.iloc[i]['Radius']
        if math.isnan(dv1r): #if no daughters, skip
            area = np.append(area,np.nan)
            continue
        area_value = (dv1r**2+dv2r**2)/(pv**2) #calculate eta
        area = np.append(area,area_value)
    data['Eta'] = area.tolist() #add to table
    return data

def angleExtractor(name,pruned):
    if pruned == 0:
        file_path = 'Networks/Network_Vessels_' + name +'.mat'
    elif pruned == 1: 
        file_path = 'Pruned/Pruned_Network_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    angles = matlab_data['Angles']
    headers = [str(h.item()) if isinstance(h,np.ndarray) else str(h) for h in angles[0]]
    columns = [angles[1,i].squeeze() for i in range(len(headers))]
    angled = pd.DataFrame(dict(zip(headers,columns)))
    return angled

def matchAngles(data,name,pruned):
    angles = angleExtractor(name,pruned)
    edges = edgesExtractor(name,pruned)
    maps = mapIDExtractor(name,pruned)
    newOldIDs = pd.merge(maps, edges, on='ID', how='left')
    angleID = pd.merge(newOldIDs,angles,left_on='To',right_on='hubNode')
    angleID.drop(['ID','From','To','hubNode'],axis=1,inplace=True)
    #vesAngle = angleID.merge(data,left_on='New',right_on='Parent',how='left')
    angled = data.merge(angleID,left_on='Parent',right_on='New',how='left')
    angled = angled.drop(columns='New')
    return angled

def offPrincipalPath(data):
    # Initialize BFS queue and results
    queue = deque([0])  # Start with the first row
    visited = set()
    principal_path = []
    
    # Get the initial radius
    root_radius = data.loc[0, 'Radius']
    
    while queue:
        current = queue.popleft()
        if current in visited:
            continue
        visited.add(current)
        principal_path.append(current)
        
        # Parent radius
        parent_radius = data.loc[current, 'Radius']
        
        # Check each daughter
        for i in range(1, 4):
            daughter_col = f'Daughter{i}'
            dvr_col = f'DV{i}R'
            if daughter_col in data.columns and dvr_col in data.columns:
                daughter = data.loc[current, daughter_col]
                dvr = data.loc[current, dvr_col]
                if not pd.isna(daughter) and not pd.isna(dvr):
                    if dvr >= 0.4 * root_radius:
                        queue.append(int(daughter))
    
    # Create a copy of the data
    result_df = data.copy()
    
    # Set specified columns to NaN for vessels on the principal pathway
    columns_to_nan = ['Alpha', 'Beta', 'Gamma', 'Eta', 'Xi']
    result_df.loc[result_df['Parent'].isin(principal_path), columns_to_nan] = np.nan
    
    return result_df

In [18]:
def dataExtractor(name,pruned): #sequentially calls necessary functions to create full table
    connectivity = connectivityExtractor(name,pruned)
    length = lengthExtractor(name,pruned)
    radius = radiusExtractor(name,pruned)
    volume = volumeExtractor(name,pruned)
    combined = pd.concat([connectivity.iloc[:, :1], length, radius, volume, connectivity.iloc[:, 1:]], axis=1)
    lengthradius = lRR(combined)
    #print('LRR')
    depthness = assignDepth(lengthradius)
    #print('Depth')
    lengthTotal = lengthFromRoot(depthness)
    #print('LengthFromRoot')
    tortuos = tortuosityCalculator(lengthTotal,name,pruned)
    #print('Tortousity')
    allradii = radiusFinder(tortuos)
    angled = matchAngles(allradii,name,pruned)
    alphabetazed = alphaBeta(angled)
    #print('Alpha/beta')
    xi = xiFinder(alphabetazed)
    #print('Xi')
    asymmetric = asymmetry(xi)
    #print('Gamma')
    areated = area(asymmetric)
    opp = offPrincipalPath(areated)
    return opp

def saveStatistics(name,pruned): #saves the stats as .csv for exporting
    statistics = dataExtractor(name,pruned)
    file_path = 'Statistics/Statistics_' + name +'.csv'
    statistics.to_csv(file_path) #shouldn't return anything, just save

In [19]:
def lobeExtractor(name, vesID, pruned):
    data = dataExtractor(name, pruned)
    tree = defaultdict(list)
    for _, row in data.iterrows():
        parent = row['Parent']
        for daughter_col in ['Daughter1', 'Daughter2', 'Daughter3']:
            daughter = row[daughter_col]
            if pd.notna(daughter):
                tree[parent].append(daughter)
    
    visited = set()
    queue = deque([vesID])
    while queue:
        current = queue.popleft()
        if current not in visited:
            visited.add(current)
            queue.extend(tree.get(current, []))
    
    downstream_df = data[data['Parent'].isin(visited) | data['Parent'].isin([vesID])]
    
    # Get the input vessel's values for Depth and LengthFromRoot
    input_vessel_row = downstream_df[downstream_df['Parent'] == vesID]
    
    input_depth = input_vessel_row['Depth'].iloc[0]
    input_length = input_vessel_row['LengthFromRoot'].iloc[0]
    
    # Subtract the input vessel's values from all rows
    downstream_df['Depth'] = downstream_df['Depth'] - input_depth
    downstream_df['LengthFromRoot'] = downstream_df['LengthFromRoot'] - input_length
    
    return downstream_df.reset_index(drop=True)


def lobeInput(name,fromnode,tonode,pruned):
    segments = edgesExtractor(name,pruned)
    maps = mapIDExtractor(name,pruned)
    newOldIDs = pd.merge(maps, segments, on='ID', how='left')
    newVesID = int(newOldIDs[((newOldIDs['From'] == fromnode)&(newOldIDs['To']==tonode))].iloc[0,0])
    lobe = lobeExtractor(name,newVesID,pruned)
    return lobe

def saveLobeStatistics(name,fromnode,tonode,pruned,lobe,pressure):
    lobe_stats = lobeInput(name,fromnode,tonode,pruned)
    file_path = 'lobeStatistics/Pressure'+pressure+'/'+lobe+'/'+name+'_Pressure'+pressure+'_'+lobe+'.csv'
    lobe_stats.to_csv(file_path) #shouldn't return anything, just save

def saveTwoInputs(name,from1,to1,from2,to2,pruned,lobe,pressure):
    lobe_stats1 = lobeInput(name,from1,to1,pruned)
    lobe_stats2 = lobeInput(name,from2,to2,pruned)
    combined_lobe_stats = pd.concat([lobe_stats1, lobe_stats2]).reset_index(drop=True)
    file_path = 'lobeStatistics/Pressure'+pressure+'/'+lobe+'/'+name+'_Pressure'+pressure+'_'+lobe+'.csv'
    combined_lobe_stats.to_csv(file_path) #shouldn't return anything, just save

In [20]:
for pressure in ['1','2','3','4']:
    lobe = 'middle'
    if lobe == "middle":
        if pressure == '1':
            m1053007 = [1841, 1945, 1841, 1964]
            m2053007 = [2463, 2497, 2502, 2501]
            m1053107 = [4179, 4304, 4018, 4379]
            m2053107 = [2920, 2921, 3150, 3149]
            m1060107 = [2766, 2744, 2992, 1332]
            m1060407 = [2456, 2459, 2455, 1061]
            m2060407 = [2963, 2568, 2964, 2995]
            m3060407 = [2509, 2613, 2507, 408]
            m1060507 = [1992, 1929, 2030, 2065]
            m2060507 = [2390, 2227, 2284, 2283]
            m3060507 = [2624, 2377, 2568, 1365]
            m2060607 = [1916, 2052, 2057, 1991]
            m3060607 = [2604, 2466, 2677, 2607]
        if pressure == '2':
            m1053007 = [1841, 1957, 1841, 2025]
            m2053007 = [1831, 1854, 1908, 1919]
            m1053107 = [2738, 2739, 2959, 1567]
            m2053107 = [1645, 1644, 1642, 1641]
            m1060107 = [2164, 2098, 2153, 927]
            m1060407 = [1289, 1269, 1253, 1252]
            m2060407 = [1464, 1466, 1465, 592]
            m3060407 = [1907, 1962, 1939, 337]
            m1060507 = [1322, 1400, 1320, 1425]
            m2060507 = [1565, 1566, 1578, 1577]
            m3060507 = [1732, 1589, 1749, 1109]
            m2060607 = [1441, 1221, 1460, 221]
            m3060607 = [1925, 1946, 2002, 2094]
        if pressure == '3':
            m1053007 = [1721, 1720, 1721, 95]
            m2053007 = [1731, 1737, 1682, 1733]
            m1053107 = [3103, 2859, 2892, 844]
            m2053107 = [1621, 1622, 1714, 1709]
            m1060107 = [1651, 1727, 1611, 1610]
            m1060407 = [1110, 1109, 1140, 426]
            m2060407 = [854, 283, 891, 50]
            m3060407 = [968, 938, 967, 976]
            m1060507 = [1470, 1475, 1469, 1370]
            m2060507 = [1041, 1121, 1042, 1163]
            m3060507 = [1000, 451, 1062, 1016]
            m2060607 = [607, 595, 616, 640]
            m3060607 = [1261, 1202, 1257, 1268]
        if pressure == '4':
            m1053007 = [1954, 1803, 1954, 1788]
            m2053007 = [1223, 161, 1273, 1342]
            m1053107 = [2059, 2186, 2058, 1054]
            m2053107 = [1520, 1532, 1555, 1565]
            m1060107 = [1652, 1619, 1653, 1529]
            m1060407 = [1055, 1054, 1084, 516]
            m2060407 = [692, 127, 690, 313]
            m3060407 = [865, 61, 878, 880]
            m1060507 = [1243, 1244, 1212, 1341]
            m2060507 = [944, 993, 945, 969]
            m3060507 = [959, 960, 1003, 1093]
            m2060607 = [753, 768, 720, 719]
            m3060607 = [1290, 270, 1207, 1309]
    if lobe == 'middle':
        datasets = {
        'm1053007': saveTwoInputs('m1p'+pressure+'_053007',m1053007[0], m1053007[1], m1053007[2], m1053007[3], 1,lobe,pressure),
        'm2053007': saveTwoInputs('m2p'+pressure+'_053007',m2053007[0], m2053007[1], m2053007[2], m2053007[3], 1,lobe,pressure),
        'm1053107': saveTwoInputs('m1p'+pressure+'_053107',m1053107[0], m1053107[1], m1053107[2], m1053107[3], 1,lobe,pressure),
        'm2053107': saveTwoInputs('m2p'+pressure+'_053107',m2053107[0], m2053107[1], m2053107[2], m2053107[3], 1,lobe,pressure),
        'm1060107': saveTwoInputs('m1p'+pressure+'_060107',m1060107[0], m1060107[1], m1060107[2], m1060107[3], 1,lobe,pressure),
        'm1060407': saveTwoInputs('m1p'+pressure+'_060407',m1060407[0], m1060407[1], m1060407[2], m1060407[3], 0,lobe,pressure),
        'm2060407': saveTwoInputs('m2p'+pressure+'_060407',m2060407[0], m2060407[1], m2060407[2], m2060407[3], 0,lobe,pressure),
        'm3060407': saveTwoInputs('m3p'+pressure+'_060407',m3060407[0], m3060407[1], m3060407[2], m3060407[3], 0,lobe,pressure),
        'm1060507': saveTwoInputs('m1p'+pressure+'_060507',m1060507[0], m1060507[1], m1060507[2], m1060507[3], 0,lobe,pressure),
        'm2060507': saveTwoInputs('m2p'+pressure+'_060507',m2060507[0], m2060507[1], m2060507[2], m2060507[3], 0,lobe,pressure),
        'm3060507': saveTwoInputs('m3p'+pressure+'_060507',m3060507[0], m3060507[1], m3060507[2], m3060507[3], 0,lobe,pressure),
        'm2060607': saveTwoInputs('m2p'+pressure+'_060607',m2060607[0], m2060607[1], m2060607[2], m2060607[3], 0,lobe,pressure),
        'm3060607': saveTwoInputs('m3p'+pressure+'_060607',m3060607[0], m3060607[1], m3060607[2], m3060607[3], 0,lobe,pressure),
        }
    
        

In [21]:
pressure = '1'
for lobe in ['left','inferior','superior','postcaval']:
    if lobe == 'left':
        m1053007 = [1831, 1858]
        m2053007 = [2367, 2368]
        m1053107 = [3954, 3900]
        m2053107 = [2866, 2868]
        m1060107 = [190, 2722]
        m1060407 = [2272, 2273]
        m2060407 = [2776, 2774]
        m3060407 = [2473, 2472]
        m1060507 = [2121, 2040]
        m2060507 = [2257, 2258]
        m3060507 = [692, 2524]
        m2060607 = [1475, 1997]
        m3060607 = [53, 2576]
    if lobe == 'superior':
        m1053007 = [1836, 1835]
        m2053007 = [2464, 2406]
        m1053107 = [4071, 685]
        m2053107 = [2867, 2979]
        m1060107 = [2780, 2716, 2780, 2781]
        m1060407 = [2274, 2283]
        m2060407 = [2742, 2598]
        m3060407 = [2418, 2419, 2420, 2491]
        m1060507 = [1993, 1997]
        m2060507 = [2259, 2330]
        m3060507 = [2392, 2398]
        m2060607 = [1914, 1915]
        m3060607 = [2603, 2518]
    if lobe == 'inferior':
        m1053007 = [1841, 1839]
        m2053007 = [2502, 2501]
        m1053107 = [4018, 4019]
        m2053107 = [3150, 3170]
        m1060107 = [2992, 2895]
        m1060407 = [2455, 2394]
        m2060407 = [2964, 2727]
        m3060407 = [2507, 2508]
        m1060507 = [2030, 2027]
        m2060507 = [2391, 2423]
        m3060507 = [2537, 1693]
        m2060607 = [2057, 2056]
        m3060607 = [2677, 2676]
    if lobe == 'postcaval':
        m1053007 = [1864, 35]
        m2053007 = [2465, 2558]
        m1053107 = [4296, 1851]
        m2053107 = [2915, 2913]
        m1060107 = [2766, 2808]
        m1060407 = [2454, 2602]
        m2060407 = [2886, 2962]
        m3060407 = [2421, 2469]
        m1060507 = [1994, 2178]
        m2060507 = [2284, 2454]
        m3060507 = [2628, 2355]
        m2060607 = [1916, 1970]
        m3060607 = [2701, 2688]
    if lobe == 'superior':
        datasets = {
        'm1053007': saveLobeStatistics('m1p'+pressure+'_053007',m1053007[0], m1053007[1], 1,lobe,pressure),
        'm2053007': saveLobeStatistics('m2p'+pressure+'_053007',m2053007[0], m2053007[1], 1,lobe,pressure),
        'm1053107': saveLobeStatistics('m1p'+pressure+'_053107',m1053107[0], m1053107[1], 1,lobe,pressure),
        'm2053107': saveLobeStatistics('m2p'+pressure+'_053107',m2053107[0], m2053107[1], 1,lobe,pressure),
        'm1060107': saveTwoInputs('m1p'+pressure+'_060107',m1060107[0], m1060107[1], m1060107[2], m1060107[3], 1,lobe,pressure),
        'm1060407': saveLobeStatistics('m1p'+pressure+'_060407',m1060407[0], m1060407[1], 0,lobe,pressure),
        'm2060407': saveLobeStatistics('m2p'+pressure+'_060407',m2060407[0], m2060407[1], 0,lobe,pressure),
        'm3060407': saveTwoInputs('m3p'+pressure+'_060407',m3060407[0], m3060407[1], m3060407[2], m3060407[3], 0,lobe,pressure),
        'm1060507': saveLobeStatistics('m1p'+pressure+'_060507',m1060507[0], m1060507[1], 0,lobe,pressure),
        'm2060507': saveLobeStatistics('m2p'+pressure+'_060507',m2060507[0], m2060507[1], 0,lobe,pressure),
        'm3060507': saveLobeStatistics('m3p'+pressure+'_060507',m3060507[0], m3060507[1], 0,lobe,pressure),
        'm2060607': saveLobeStatistics('m2p'+pressure+'_060607',m2060607[0], m2060607[1], 0,lobe,pressure),
        'm3060607': saveLobeStatistics('m3p'+pressure+'_060607',m3060607[0], m3060607[1], 0,lobe,pressure),
        }
    else: 
        datasets = {
        'm1053007': saveLobeStatistics('m1p'+pressure+'_053007',m1053007[0], m1053007[1], 1,lobe,pressure),
        'm2053007': saveLobeStatistics('m2p'+pressure+'_053007',m2053007[0], m2053007[1], 1,lobe,pressure),
        'm1053107': saveLobeStatistics('m1p'+pressure+'_053107',m1053107[0], m1053107[1], 1,lobe,pressure),
        'm2053107': saveLobeStatistics('m2p'+pressure+'_053107',m2053107[0], m2053107[1], 1,lobe,pressure),
        'm1060107': saveLobeStatistics('m1p'+pressure+'_060107',m1060107[0], m1060107[1], 1,lobe,pressure),
        'm1060407': saveLobeStatistics('m1p'+pressure+'_060407',m1060407[0], m1060407[1], 0,lobe,pressure),
        'm2060407': saveLobeStatistics('m2p'+pressure+'_060407',m2060407[0], m2060407[1], 0,lobe,pressure),
        'm3060407': saveLobeStatistics('m3p'+pressure+'_060407',m3060407[0], m3060407[1], 0,lobe,pressure),
        'm1060507': saveLobeStatistics('m1p'+pressure+'_060507',m1060507[0], m1060507[1], 0,lobe,pressure),
        'm2060507': saveLobeStatistics('m2p'+pressure+'_060507',m2060507[0], m2060507[1], 0,lobe,pressure),
        'm3060507': saveLobeStatistics('m3p'+pressure+'_060507',m3060507[0], m3060507[1], 0,lobe,pressure),
        'm2060607': saveLobeStatistics('m2p'+pressure+'_060607',m2060607[0], m2060607[1], 0,lobe,pressure),
        'm3060607': saveLobeStatistics('m3p'+pressure+'_060607',m3060607[0], m3060607[1], 0,lobe,pressure),
        }
    

In [22]:
pressure = '2'
for lobe in ['left','inferior','superior','postcaval']:
    if lobe == 'left':
        m1053007 = [1075, 1960]
        m2053007 = [217, 1868]
        m1053107 = [1506, 2897]
        m2053107 = [44, 1616]
        m1060107 = [2089, 2169]
        m1060407 = [1225, 1265]
        m2060407 = [1371, 1392]
        m3060407 = [1882, 1924]
        m1060507 = [118, 1381]
        m2060507 = [1509, 1519]
        m3060507 = [51, 1591]
        m2060607 = [1495, 1419]
        m3060607 = [535, 1916]
    if lobe == 'superior':
        m1053007 = [1829, 1828]
        m2053007 = [1829, 1830]
        m1053107 = [2909, 2895]
        m2053107 = [1640, 1693]
        m1060107 = [2162, 2163]
        m1060407 = [1266, 1239]
        m2060407 = [1387, 1405]
        m3060407 = [1883, 1890, 1880, 1881]
        m1060507 = [1468, 1454]
        m2060507 = [1528, 1552]
        m3060507 = [1706, 1705]
        m2060607 = [1440, 1426]
        m3060607 = [1923, 1924]
    if lobe == 'inferior':
        m1053007 = [1842, 2020]
        m2053007 = [1908, 1979]
        m1053107 = [2959, 2957]
        m2053107 = [1642, 1755]
        m1060107 = [2153, 2151]
        m1060407 = [1253, 1252]
        m2060407 = [1465, 1515]
        m3060407 = [1939, 1982]
        m1060507 = [1320, 1424]
        m2060507 = [1620, 1602]
        m3060507 = [1749, 1809]
        m2060607 = [1460, 1375]
        m3060607 = [2002, 1903]
    if lobe == 'postcaval':
        m1053007 = [1862, 2003]
        m2053007 = [1907, 1909]
        m1053107 = [3056, 1829]
        m2053107 = [1724, 1722]
        m1060107 = [2164, 2190]
        m1060407 = [1290, 1323]
        m2060407 = [1386, 1410]
        m3060407 = [1907, 1796]
        m1060507 = [1319, 1321]
        m2060507 = [1578, 1650]
        m3060507 = [1739, 1689]
        m2060607 = [1441, 1509]
        m3060607 = [1951, 1950]
    if lobe == 'superior':
        datasets = {
        'm1053007': saveLobeStatistics('m1p'+pressure+'_053007',m1053007[0], m1053007[1], 1,lobe,pressure),
        'm2053007': saveLobeStatistics('m2p'+pressure+'_053007',m2053007[0], m2053007[1], 1,lobe,pressure),
        'm1053107': saveLobeStatistics('m1p'+pressure+'_053107',m1053107[0], m1053107[1], 1,lobe,pressure),
        'm2053107': saveLobeStatistics('m2p'+pressure+'_053107',m2053107[0], m2053107[1], 1,lobe,pressure),
        'm1060107': saveLobeStatistics('m1p'+pressure+'_060107',m1060107[0], m1060107[1], 1,lobe,pressure),
        'm1060407': saveLobeStatistics('m1p'+pressure+'_060407',m1060407[0], m1060407[1], 0,lobe,pressure),
        'm2060407': saveLobeStatistics('m2p'+pressure+'_060407',m2060407[0], m2060407[1], 0,lobe,pressure),
        'm3060407': saveTwoInputs('m3p'+pressure+'_060407',m3060407[0], m3060407[1], m3060407[2], m3060407[3], 0,lobe,pressure),
        'm1060507': saveLobeStatistics('m1p'+pressure+'_060507',m1060507[0], m1060507[1], 0,lobe,pressure),
        'm2060507': saveLobeStatistics('m2p'+pressure+'_060507',m2060507[0], m2060507[1], 0,lobe,pressure),
        'm3060507': saveLobeStatistics('m3p'+pressure+'_060507',m3060507[0], m3060507[1], 0,lobe,pressure),
        'm2060607': saveLobeStatistics('m2p'+pressure+'_060607',m2060607[0], m2060607[1], 0,lobe,pressure),
        'm3060607': saveLobeStatistics('m3p'+pressure+'_060607',m3060607[0], m3060607[1], 0,lobe,pressure),
        }
    else: 
        datasets = {
        'm1053007': saveLobeStatistics('m1p'+pressure+'_053007',m1053007[0], m1053007[1], 1,lobe,pressure),
        'm2053007': saveLobeStatistics('m2p'+pressure+'_053007',m2053007[0], m2053007[1], 1,lobe,pressure),
        'm1053107': saveLobeStatistics('m1p'+pressure+'_053107',m1053107[0], m1053107[1], 1,lobe,pressure),
        'm2053107': saveLobeStatistics('m2p'+pressure+'_053107',m2053107[0], m2053107[1], 1,lobe,pressure),
        'm1060107': saveLobeStatistics('m1p'+pressure+'_060107',m1060107[0], m1060107[1], 1,lobe,pressure),
        'm1060407': saveLobeStatistics('m1p'+pressure+'_060407',m1060407[0], m1060407[1], 0,lobe,pressure),
        'm2060407': saveLobeStatistics('m2p'+pressure+'_060407',m2060407[0], m2060407[1], 0,lobe,pressure),
        'm3060407': saveLobeStatistics('m3p'+pressure+'_060407',m3060407[0], m3060407[1], 0,lobe,pressure),
        'm1060507': saveLobeStatistics('m1p'+pressure+'_060507',m1060507[0], m1060507[1], 0,lobe,pressure),
        'm2060507': saveLobeStatistics('m2p'+pressure+'_060507',m2060507[0], m2060507[1], 0,lobe,pressure),
        'm3060507': saveLobeStatistics('m3p'+pressure+'_060507',m3060507[0], m3060507[1], 0,lobe,pressure),
        'm2060607': saveLobeStatistics('m2p'+pressure+'_060607',m2060607[0], m2060607[1], 0,lobe,pressure),
        'm3060607': saveLobeStatistics('m3p'+pressure+'_060607',m3060607[0], m3060607[1], 0,lobe,pressure),
        }
    

In [23]:
pressure = '3'
for lobe in ['left','inferior','superior','postcaval']:
    if lobe == 'left':
        m1053007 = [1630, 1628]
        m2053007 = [1619, 1621]
        m1053107 = [246, 2864]
        m2053107 = [717, 1751]
        m1060107 = [318, 1645]
        m1060407 = [1122, 1121]
        m2060407 = [808, 814]
        m3060407 = [919, 921]
        m1060507 = [80, 1354]
        m2060507 = [1097, 1116]
        m3060507 = [191, 959]
        m2060607 = [653, 620]
        m3060607 = [142, 1171]
    if lobe == 'superior':
        m1053007 = [1657, 1656]
        m2053007 = [1620, 1703]
        m1053107 = [2982, 2955]
        m2053107 = [1612, 1610]
        m1060107 = [1644, 1650]
        m1060407 = [1126, 1095]
        m2060407 = [862, 832]
        m3060407 = [969, 970, 920, 924] 
        m1060507 = [1405, 1461]
        m2060507 = [1096, 1095]
        m3060507 = [952, 950]
        m2060607 = [615, 642]
        m3060607 = [1172, 1255]
    if lobe == 'inferior':
        m1053007 = [1721, 1624]
        m2053007 = [1682, 1683]
        m1053107 = [2892, 1313]
        m2053107 = [1714, 1715]
        m1060107 = [1611, 1757]
        m1060407 = [1140, 1157]
        m2060407 = [891, 844]
        m3060407 = [967, 975]
        m1060507 = [1469, 1488]
        m2060507 = [1040, 1043]
        m3060507 = [1062, 1063]
        m2060607 = [616, 637]
        m3060607 = [1257, 1284]
    if lobe == 'postcaval':
        m1053007 = [1755, 1636]
        m2053007 = [1704, 1732]
        m1053107 = [3039, 1474]
        m2053107 = [1655, 1653]
        m1060107 = [1651, 29]
        m1060407 = [1134, 1141]
        m2060407 = [851, 853]
        m3060407 = [968, 1020]
        m1060507 = [1381, 1379]
        m2060507 = [1042, 1136]
        m3060507 = [1009, 1057]
        m2060607 = [607, 664]
        m3060607 = [1250, 1249]
    if lobe == 'superior':
        datasets = {
        'm1053007': saveLobeStatistics('m1p'+pressure+'_053007',m1053007[0], m1053007[1], 1,lobe,pressure),
        'm2053007': saveLobeStatistics('m2p'+pressure+'_053007',m2053007[0], m2053007[1], 1,lobe,pressure),
        'm1053107': saveLobeStatistics('m1p'+pressure+'_053107',m1053107[0], m1053107[1], 1,lobe,pressure),
        'm2053107': saveLobeStatistics('m2p'+pressure+'_053107',m2053107[0], m2053107[1], 1,lobe,pressure),
        'm1060107': saveLobeStatistics('m1p'+pressure+'_060107',m1060107[0], m1060107[1], 1,lobe,pressure),
        'm1060407': saveLobeStatistics('m1p'+pressure+'_060407',m1060407[0], m1060407[1], 0,lobe,pressure),
        'm2060407': saveLobeStatistics('m2p'+pressure+'_060407',m2060407[0], m2060407[1], 0,lobe,pressure),
        'm3060407': saveTwoInputs('m3p'+pressure+'_060407',m3060407[0], m3060407[1], m3060407[2], m3060407[3], 0,lobe,pressure),
        'm1060507': saveLobeStatistics('m1p'+pressure+'_060507',m1060507[0], m1060507[1], 0,lobe,pressure),
        'm2060507': saveLobeStatistics('m2p'+pressure+'_060507',m2060507[0], m2060507[1], 0,lobe,pressure),
        'm3060507': saveLobeStatistics('m3p'+pressure+'_060507',m3060507[0], m3060507[1], 0,lobe,pressure),
        'm2060607': saveLobeStatistics('m2p'+pressure+'_060607',m2060607[0], m2060607[1], 0,lobe,pressure),
        'm3060607': saveLobeStatistics('m3p'+pressure+'_060607',m3060607[0], m3060607[1], 0,lobe,pressure),
        }
    else: 
        datasets = {
        'm1053007': saveLobeStatistics('m1p'+pressure+'_053007',m1053007[0], m1053007[1], 1,lobe,pressure),
        'm2053007': saveLobeStatistics('m2p'+pressure+'_053007',m2053007[0], m2053007[1], 1,lobe,pressure),
        'm1053107': saveLobeStatistics('m1p'+pressure+'_053107',m1053107[0], m1053107[1], 1,lobe,pressure),
        'm2053107': saveLobeStatistics('m2p'+pressure+'_053107',m2053107[0], m2053107[1], 1,lobe,pressure),
        'm1060107': saveLobeStatistics('m1p'+pressure+'_060107',m1060107[0], m1060107[1], 1,lobe,pressure),
        'm1060407': saveLobeStatistics('m1p'+pressure+'_060407',m1060407[0], m1060407[1], 0,lobe,pressure),
        'm2060407': saveLobeStatistics('m2p'+pressure+'_060407',m2060407[0], m2060407[1], 0,lobe,pressure),
        'm3060407': saveLobeStatistics('m3p'+pressure+'_060407',m3060407[0], m3060407[1], 0,lobe,pressure),
        'm1060507': saveLobeStatistics('m1p'+pressure+'_060507',m1060507[0], m1060507[1], 0,lobe,pressure),
        'm2060507': saveLobeStatistics('m2p'+pressure+'_060507',m2060507[0], m2060507[1], 0,lobe,pressure),
        'm3060507': saveLobeStatistics('m3p'+pressure+'_060507',m3060507[0], m3060507[1], 0,lobe,pressure),
        'm2060607': saveLobeStatistics('m2p'+pressure+'_060607',m2060607[0], m2060607[1], 0,lobe,pressure),
        'm3060607': saveLobeStatistics('m3p'+pressure+'_060607',m3060607[0], m3060607[1], 0,lobe,pressure),
        }
    

In [ ]:
pressure = '4'
for lobe in ['left','inferior','superior','postcaval']:
    if lobe == 'left':
        m1053007 = [1810, 1811]
        m2053007 = [1209, 1214]
        m1053107 = [1252, 2103]
        m2053107 = [1239, 1392]
        m1060107 = [215, 1664]
        m1060407 = [1065, 1013]
        m2060407 = [694, 701]
        m3060407 = [851, 926]
        m1060507 = [1202, 1169]
        m2060507 = [1006, 1064]
        m3060507 = [395, 975]
        m2060607 = [511, 737]
        m3060607 = [1223, 1222]
    if lobe == 'superior':
        m1053007 = [1819, 1818]
        m2053007 = [1213, 1259]
        m1053107 = [2125, 2154]
        m2053107 = [1391, 1477]
        m1060107 = [1574, 1522]
        m1060407 = [1053, 1051]
        m2060407 = [695, 727]
        m3060407 = [885, 894, 884, 864]
        m1060507 = [1232, 1233]
        m2060507 = [999, 983]
        m3060507 = [1027, 1026]
        m2060607 = [738, 741]
        m3060607 = [1268, 1267]
    if lobe == 'inferior':
        m1053007 = [1983, 1923]
        m2053007 = [1273, 1281]
        m1053107 = [2058, 1074]
        m2053107 = [1555, 1494]
        m1060107 = [1653, 1687]
        m1060407 = [1084, 1098]
        m2060407 = [690, 691]
        m3060407 = [878, 879]
        m1060507 = [1212, 1213]
        m2060507 = [943, 946]
        m3060507 = [1003, 1005]
        m2060607 = [720, 769]
        m3060607 = [1207, 1205]
    if lobe == 'postcaval':
        m1053007 = [1903, 511]
        m2053007 = [1224, 1361]
        m1053107 = [2057, 1273]
        m2053107 = [1550, 1556]
        m1060107 = [1652, 1713]
        m1060407 = [1059, 1083]
        m2060407 = [783, 739]
        m3060407 = [865, 920]
        m1060507 = [1234, 1326]
        m2060507 = [945, 1036]
        m3060507 = [986, 985]
        m2060607 = [753, 752]
        m3060607 = [1207, 1274]
    if lobe == 'superior':
        datasets = {
        'm1053007': saveLobeStatistics('m1p'+pressure+'_053007',m1053007[0], m1053007[1], 1,lobe,pressure),
        'm2053007': saveLobeStatistics('m2p'+pressure+'_053007',m2053007[0], m2053007[1], 1,lobe,pressure),
        'm1053107': saveLobeStatistics('m1p'+pressure+'_053107',m1053107[0], m1053107[1], 1,lobe,pressure),
        'm2053107': saveLobeStatistics('m2p'+pressure+'_053107',m2053107[0], m2053107[1], 1,lobe,pressure),
        'm1060107': saveLobeStatistics('m1p'+pressure+'_060107',m1060107[0], m1060107[1], 1,lobe,pressure),
        'm1060407': saveLobeStatistics('m1p'+pressure+'_060407',m1060407[0], m1060407[1], 0,lobe,pressure),
        'm2060407': saveLobeStatistics('m2p'+pressure+'_060407',m2060407[0], m2060407[1], 0,lobe,pressure),
        'm3060407': saveTwoInputs('m3p'+pressure+'_060407',m3060407[0], m3060407[1], m3060407[2], m3060407[3], 0,lobe,pressure),
        'm1060507': saveLobeStatistics('m1p'+pressure+'_060507',m1060507[0], m1060507[1], 0,lobe,pressure),
        'm2060507': saveLobeStatistics('m2p'+pressure+'_060507',m2060507[0], m2060507[1], 0,lobe,pressure),
        'm3060507': saveLobeStatistics('m3p'+pressure+'_060507',m3060507[0], m3060507[1], 0,lobe,pressure),
        'm2060607': saveLobeStatistics('m2p'+pressure+'_060607',m2060607[0], m2060607[1], 0,lobe,pressure),
        'm3060607': saveLobeStatistics('m3p'+pressure+'_060607',m3060607[0], m3060607[1], 0,lobe,pressure),
        }
    else: 
        datasets = {
        'm1053007': saveLobeStatistics('m1p'+pressure+'_053007',m1053007[0], m1053007[1], 1,lobe,pressure),
        'm2053007': saveLobeStatistics('m2p'+pressure+'_053007',m2053007[0], m2053007[1], 1,lobe,pressure),
        'm1053107': saveLobeStatistics('m1p'+pressure+'_053107',m1053107[0], m1053107[1], 1,lobe,pressure),
        'm2053107': saveLobeStatistics('m2p'+pressure+'_053107',m2053107[0], m2053107[1], 1,lobe,pressure),
        'm1060107': saveLobeStatistics('m1p'+pressure+'_060107',m1060107[0], m1060107[1], 1,lobe,pressure),
        'm1060407': saveLobeStatistics('m1p'+pressure+'_060407',m1060407[0], m1060407[1], 0,lobe,pressure),
        'm2060407': saveLobeStatistics('m2p'+pressure+'_060407',m2060407[0], m2060407[1], 0,lobe,pressure),
        'm3060407': saveLobeStatistics('m3p'+pressure+'_060407',m3060407[0], m3060407[1], 0,lobe,pressure),
        'm1060507': saveLobeStatistics('m1p'+pressure+'_060507',m1060507[0], m1060507[1], 0,lobe,pressure),
        'm2060507': saveLobeStatistics('m2p'+pressure+'_060507',m2060507[0], m2060507[1], 0,lobe,pressure),
        'm3060507': saveLobeStatistics('m3p'+pressure+'_060507',m3060507[0], m3060507[1], 0,lobe,pressure),
        'm2060607': saveLobeStatistics('m2p'+pressure+'_060607',m2060607[0], m2060607[1], 0,lobe,pressure),
        'm3060607': saveLobeStatistics('m3p'+pressure+'_060607',m3060607[0], m3060607[1], 0,lobe,pressure),
        }
   

: 